In [1]:
import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import h5py
import napari
import seaborn as sns

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()

In [5]:
import PPIGraph

# Read per cell data

In [6]:
# Read PPI dfs
PPI_save_path =  data_dir / '9PPI' / 'PPI'

dfs = []
for path in os.listdir(PPI_save_path):
    if 'csv' in path:
        df = pd.read_csv(PPI_save_path / path)
        dfs.append(df)

df = pd.concat(dfs)

# Group by location (Cell vs Nuclei)
g = df.groupby(['Condition', 'FOV', 'PPI', 'Cyto']).size()
df_cell = pd.DataFrame({'Count Cyto': g}).reset_index()
df_cell = df_cell[df_cell.Cyto != 0]
df_cell.columns = ['Condition', 'FOV', 'PPI', 'Id', 'Count_cell']

g = df.groupby(['Condition', 'FOV', 'PPI', 'Nuclei']).size()
df_nuclei = pd.DataFrame({'Count Nuclei': g}).reset_index()
df_nuclei= df_nuclei[df_nuclei.Nuclei != 0]
df_nuclei.columns = ['Condition', 'FOV', 'PPI', 'Id', 'Count_nuclei']

# Concat 
df_all = df_cell.merge(df_nuclei, how='left', on=['Condition', 'FOV', 'PPI', 'Id']).fillna(0)
df_all['Count_nuclei'] = df_all['Count_nuclei'].astype(int)
df_all['Count_cyto'] = df_all['Count_cell'] - df_all['Count_nuclei'] # Cyto count

# Ranme columns
df_all.columns = ['Condition', 'FOV', 'PPI', 'Id', 'Cell', 'Nuclei',
       'Cyto']

# Plot descrition
display(df_all.describe())
display(df_all.groupby(['Condition', 'FOV', 'Id']).sum().describe())

,Id,Cell,Nuclei,Cyto
count,12403.000000,12403.000000,12403.000000,12403.000000
mean,205.029993,47.493832,16.249456,31.244376
std,123.872364,81.222652,24.820231,60.451342
min,5.000000,1.000000,0.000000,-19.000000
25%,99.000000,4.000000,1.000000,2.000000
50%,197.000000,20.000000,6.000000,11.000000
75%,298.000000,51.000000,20.000000,31.000000
max,519.000000,1661.000000,265.000000,1439.000000


C:\Users\thu71\AppData\Local\Temp\ipykernel_16740\2680352089.py:34: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  display(df_all.groupby(['Condition', 'FOV', 'Id']).sum().describe())


,Cell,Nuclei,Cyto
count,1491.000000,1491.000000,1491.000000
mean,395.081154,135.172368,259.908786
std,221.626206,67.238884,179.832845
min,48.000000,0.000000,-26.000000
25%,244.000000,95.000000,137.000000
50%,349.000000,127.000000,222.000000
75%,491.000000,165.000000,334.000000
max,2728.000000,755.000000,2233.000000


In [7]:
# Filter out by maximum number of counts per cell
min_count = 100
max_count = 400

df_all = df_all.groupby(['Condition', 'FOV', 'Id']).filter(lambda x: x['Cell'].sum() > min_count)
df_all = df_all.groupby(['Condition', 'FOV', 'Id']).filter(lambda x: (x['Cell'] < max_count).all())
df_all = df_all.groupby(['Condition', 'FOV', 'Id']).filter(lambda x: (x['Cyto'] >= 0).all())

# Plot descrition
display(df_all.describe())
display(df_all.groupby(['Condition', 'FOV', 'Id']).sum().describe())

,Id,Cell,Nuclei,Cyto
count,11057.000000,11057.000000,11057.000000,11057.000000
mean,205.392602,42.808628,15.386000,27.422628
std,122.013801,65.600235,22.467235,46.563063
min,5.000000,1.000000,0.000000,0.000000
25%,103.000000,4.000000,1.000000,2.000000
50%,198.000000,20.000000,6.000000,11.000000
75%,296.000000,48.000000,20.000000,29.000000
max,519.000000,399.000000,183.000000,369.000000


C:\Users\thu71\AppData\Local\Temp\ipykernel_16740\1132281257.py:11: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  display(df_all.groupby(['Condition', 'FOV', 'Id']).sum().describe())


,Cell,Nuclei,Cyto
count,1334.000000,1334.000000,1334.000000
mean,354.823838,127.528486,227.295352
std,147.709452,52.618519,121.642145
min,101.000000,0.000000,28.000000
25%,241.250000,94.000000,135.000000
50%,333.000000,124.000000,209.000000
75%,451.750000,158.000000,302.000000
max,1202.000000,423.000000,1202.000000


# ML model

In [8]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing, metrics
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split, KFold
from sklearn.utils.class_weight import compute_class_weight
import wandb

In [9]:
condition = 'ML'
project_name = f'PLA_9PPI_121923_{condition}'

In [10]:
# Get dataframe seperate into count per cell 
df_count = df_all.pivot(index=['Condition', 'FOV', 'Id'], columns=['PPI']).fillna(0)
df_count_cell = df_count.iloc[:, df_count.columns.get_level_values(0)=='Cell']

# Get dataframe seperate into count per nuclei and cyto
df_count_cyto_nuclei = df_count.iloc[:, df_count.columns.get_level_values(0)!='Cell']

# Get Condition into numerical label
le = preprocessing.LabelEncoder()
y = le.fit_transform(df_count.index.get_level_values(0))

In [11]:
df_count

Cell                                         \
PPI                Bim & Tom20 Cyclin D1 & CDK2 Cyclin D1 & CDK2 - re   
Condition  FOV Id                                                       
HCC827Ctrl FW1 15         20.0            185.0                  14.0   
               16          7.0            103.0                  26.0   
               17         23.0            121.0                  19.0   
               18         57.0            225.0                  11.0   
               19         26.0            139.0                  31.0   
...                        ...              ...                   ...   
HCC827Osim FW2 340        39.0             85.0                  28.0   
               341        17.0            151.0                  23.0   
               343        39.0            218.0                  15.0   
               344        22.0            206.0                  84.0   
               345        28.0            195.0                  50.0   

                                                                 \
PPI                Cyclin E & CDK4 Mcl-1 & BAK NF-Kb & p-P90rsk   
Condition  FOV Id                                                 
HCC827Ctrl FW1 15              3.0         1.0             29.0   
               16              0.0         2.0             37.0   
               17              0.0         0.0             35.0   
               18              2.0         0.0             57.0   
               19              0.0         0.0             34.0   
...                            ...         ...              ...   
HCC827Osim FW2 340             0.0         0.0             36.0   
               341             3.0         1.0             48.0   
               343             2.0         0.0             80.0   
               344             1.0         0.0             62.0   
               345             0.0         0.0             45.0   

                                                                               \
PPI                NF-Kb & p-P90rsk - re P-AKT & mTOR Sox2 & Oct4 TEAD1 & YAP   
Condition  FOV Id                                                               
HCC827Ctrl FW1 15                    0.0          2.0        39.0         3.0   
               16                    0.0          0.0        19.0         2.0   
               17                    0.0          0.0        36.0         1.0   
               18                    0.0          0.0        65.0         0.0   
               19                    0.0          0.0        39.0         0.0   
...                                  ...          ...         ...         ...   
HCC827Osim FW2 340                   0.0          0.0        37.0         0.0   
               341                   0.0          6.0        60.0         2.0   
               343                   5.0          0.0        66.0         0.0   
               344                   1.0          0.0        40.0         1.0   
               345                   0.0          0.0        58.0         0.0   

                    ...             Cyto                        \
PPI                 ... Cyclin D1 & CDK2 Cyclin D1 & CDK2 - re   
Condition  FOV Id   ...                                          
HCC827Ctrl FW1 15   ...            120.0                   2.0   
               16   ...             67.0                  13.0   
               17   ...             83.0                  12.0   
               18   ...            166.0                   8.0   
               19   ...             81.0                   2.0   
...                 ...              ...                   ...   
HCC827Osim FW2 340  ...             69.0                  28.0   
               341  ...             92.0                  12.0   
               343  ...            137.0                  13.0   
               344  ...            166.0                  48.0   
               345  ...            135.0                  33.0   

    

In [12]:
df_count.reset_index().groupby('Condition').size()

Condition
HCC827Ctrl    723
HCC827Osim    611
dtype: int64

In [13]:
df_count_cyto_nuclei

Nuclei                                         \
PPI                Bim & Tom20 Cyclin D1 & CDK2 Cyclin D1 & CDK2 - re   
Condition  FOV Id                                                       
HCC827Ctrl FW1 15          5.0             65.0                  12.0   
               16          1.0             36.0                  13.0   
               17         16.0             38.0                   7.0   
               18         40.0             59.0                   3.0   
               19         18.0             58.0                  29.0   
...                        ...              ...                   ...   
HCC827Osim FW2 340        20.0             16.0                   0.0   
               341         2.0             59.0                  11.0   
               343        11.0             81.0                   2.0   
               344        10.0             40.0                  36.0   
               345        17.0             60.0                  17.0   

                                                                 \
PPI                Cyclin E & CDK4 Mcl-1 & BAK NF-Kb & p-P90rsk   
Condition  FOV Id                                                 
HCC827Ctrl FW1 15              0.0         0.0              8.0   
               16              0.0         2.0             12.0   
               17              0.0         0.0             18.0   
               18              1.0         0.0             20.0   
               19              0.0         0.0             10.0   
...                            ...         ...              ...   
HCC827Osim FW2 340             0.0         0.0              7.0   
               341             3.0         1.0             21.0   
               343             0.0         0.0             40.0   
               344             1.0         0.0             21.0   
               345             0.0         0.0             27.0   

                                                                               \
PPI                NF-Kb & p-P90rsk - re P-AKT & mTOR Sox2 & Oct4 TEAD1 & YAP   
Condition  FOV Id                                                               
HCC827Ctrl FW1 15                    0.0          0.0        11.0         1.0   
               16                    0.0          0.0        10.0         2.0   
               17                    0.0          0.0        14.0         0.0   
               18                    0.0          0.0        39.0         0.0   
               19                    0.0          0.0        14.0         0.0   
...                                  ...          ...         ...         ...   
HCC827Osim FW2 340                   0.0          0.0         8.0         0.0   
               341                   0.0          6.0        39.0         2.0   
               343                   0.0          0.0        47.0         0.0   
               344                   0.0          0.0        20.0         1.0   
               345                   0.0          0.0        27.0         0.0   

                    ...             Cyto                        \
PPI                 ... Cyclin D1 & CDK2 Cyclin D1 & CDK2 - re   
Condition  FOV Id   ...                                          
HCC827Ctrl FW1 15   ...            120.0                   2.0   
               16   ...             67.0                  13.0   
               17   ...             83.0                  12.0   
               18   ...            166.0                   8.0   
               19   ...             81.0                   2.0   
...                 ...              ...                   ...   
HCC827Osim FW2 340  ...             69.0                  28.0   
               341  ...             92.0                  12.0   
               343  ...            137.0                  13.0   
               344  ...            166.0                  48.0   
               345  ...            135.0                  33.0   

  

In [14]:
models = {
    'Adaboost': AdaBoostClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    'GradientBoosting' : GradientBoostingClassifier(),
    'NaiveBayes': GaussianNB(),
    'RandomForest': RandomForestClassifier(), 
    'SVM': SVC(probability =True),
    'LogisticRegression':  LogisticRegression(),
    'MLP': MLPClassifier(random_state=1, max_iter=100, hidden_layer_sizes=[16, 16, 16])
}



In [15]:
# Run model on cell count
X = df_count_cell
features = ['_'.join(col) for col in X.columns.values]
X = X.values
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
kfold = KFold(n_splits = 5, shuffle = True, random_state = 0)

for model_name, model in models.items():
    for k, (train_index, test_index) in enumerate(kfold.split(X)):
        # Split the dataset
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_probas = model.predict_proba(X_test)

        run = wandb.init(project=project_name, group=model_name+'_cell', name=model_name+f'_cell_{k}')
        wandb.sklearn.plot_classifier(model, 
                                  X_train, X_test, 
                                  y_train, y_test, 
                                  y_pred, y_probas, 
                                  le.classes_, 
                                  is_binary=True, 
                                  model_name=model_name+'_cell', 
                                  feature_names=features)
        wandb.log({'roc': wandb.plots.ROC(y_test, y_probas, le.classes_)})
        wandb.log({'pr': wandb.plots.precision_recall(y_test, y_probas, le.classes_)})

        accuracy = metrics.accuracy_score(y_test, y_pred)
        b_accuracy = metrics.balanced_accuracy_score(y_test, y_pred)
        f1 = metrics.f1_score(y_test, y_pred)
        auc = metrics.roc_auc_score(y_test, y_probas[:, 1])
        wandb.log({"accuracy": accuracy, 'b_accuracy': b_accuracy, 'f1':f1, 'auc': auc})
    run.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: thoomas. Use `wandb login --relogin` to force relogin


wandb: 
wandb: Plotting Adaboost_cell.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
c:\Users\thu71\AppData\Local\anaconda3\envs\PLA2\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
wandb: Logged calibration curve.
wandb: WARNING wandb.plots.* functions are deprecated and will be removed in a future release. Please use wandb.plot.* instead.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.77903
auc,0.86265
b_accuracy,0.77955
f1,0.76305


wandb: 
wandb: Plotting Adaboost_cell.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
c:\Users\thu71\AppData\Local\anaconda3\envs\PLA2\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.77154
auc,0.8402
b_accuracy,0.77265
f1,0.76078


...

accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.8015
auc,0.83113
b_accuracy,0.78137
f1,0.72821


wandb: 
wandb: Plotting MLP_cell.
wandb: WARNING could not find any of attributes feature_importances_, feature_log_prob_, coef_ on classifier. Cannot plot feature importances.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
c:\Users\thu71\AppData\Local\anaconda3\envs\PLA2\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\thu71\AppData\Local\anaconda3\envs\PLA2\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Op

accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.77154
auc,0.84671
b_accuracy,0.76626
f1,0.7382


wandb: 
wandb: Plotting MLP_cell.
wandb: WARNING could not find any of attributes feature_importances_, feature_log_prob_, coef_ on classifier. Cannot plot feature importances.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
c:\Users\thu71\AppData\Local\anaconda3\envs\PLA2\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\thu71\AppData\Local\anaconda3\envs\PLA2\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Op

accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.75564
auc,0.82255
b_accuracy,0.75485
f1,0.74308


In [16]:
# Run model on cell count
X = df_count_cyto_nuclei
features = ['_'.join(col) for col in X.columns.values]
X = X.values
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
kfold = KFold(n_splits = 5, shuffle = True, random_state = 0)

for model_name, model in models.items():
    for k, (train_index, test_index) in enumerate(kfold.split(X)):
        # Split the dataset
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_probas = model.predict_proba(X_test)

        run = wandb.init(project=project_name, group=model_name+'_cyto_nuclei', name=model_name+f'_cyto_nuclei_{k}')
        wandb.sklearn.plot_classifier(model, 
                                  X_train, X_test, 
                                  y_train, y_test, 
                                  y_pred, y_probas, 
                                  le.classes_, 
                                  is_binary=True, 
                                  model_name=model_name+'_cyto_nuclei', 
                                  feature_names=features)
        wandb.log({'roc': wandb.plots.ROC(y_test, y_probas, le.classes_)})
        wandb.log({'pr': wandb.plots.precision_recall(y_test, y_probas, le.classes_)})

        accuracy = metrics.accuracy_score(y_test, y_pred)
        b_accuracy = metrics.balanced_accuracy_score(y_test, y_pred)
        f1 = metrics.f1_score(y_test, y_pred)
        auc = metrics.roc_auc_score(y_test, y_probas[:, 1])
        wandb.log({"accuracy": accuracy, 'b_accuracy': b_accuracy, 'f1':f1, 'auc': auc})
    run.finish()


wandb: 
wandb: Plotting Adaboost_cyto_nuclei.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
c:\Users\thu71\AppData\Local\anaconda3\envs\PLA2\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.75281
auc,0.8325
b_accuracy,0.75275
f1,0.73387


wandb: 
wandb: Plotting Adaboost_cyto_nuclei.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
c:\Users\thu71\AppData\Local\anaconda3\envs\PLA2\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.80899
auc,0.87287
b_accuracy,0.80969
f1,0.8046


wandb: 
wandb: Plotting Adaboost_cyto_nuclei.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
c:\Users\thu71\AppData\Local\anaconda3\envs\PLA2\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


...

accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.76404
auc,0.83316
b_accuracy,0.73984
f1,0.67358


wandb: 
wandb: Plotting MLP_cyto_nuclei.
wandb: WARNING could not find any of attributes feature_importances_, feature_log_prob_, coef_ on classifier. Cannot plot feature importances.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
c:\Users\thu71\AppData\Local\anaconda3\envs\PLA2\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\thu71\AppData\Local\anaconda3\envs\PLA2\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stocha

accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.71161
auc,0.82458
b_accuracy,0.70418
f1,0.66079


wandb: 
wandb: Plotting MLP_cyto_nuclei.
wandb: WARNING could not find any of attributes feature_importances_, feature_log_prob_, coef_ on classifier. Cannot plot feature importances.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=19 sec). Create a new API with an integer timeout larger than 19, e.g., `api = wandb.Api(timeout=29)` to increase the graphql timeout.
wandb: Logged class proportions.
c:\Users\thu71\AppData\Local\anaconda3\envs\PLA2\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/l

accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.76316
auc,0.82352
b_accuracy,0.7617
f1,0.74494
